In [1]:
# Libraries
from turtle import shape
# from Deblurring_NN import X_test
import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras import models
from keras import layers
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras import backend as K
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from random import randint
import zipfile
import cv2

In [2]:
# Path to input file
inputZip_blur = '/project/ctb-akhanf/myousif9/Neural_Networks_project/output/Train/deblurring.zip'
inputZip_origin = '/project/ctb-akhanf/myousif9/Neural_Networks_project/output/Train/resampled.zip'

# Path to output dir
saveDir = '../output/Test/' # was called path

# Zip file to save test cases
outZip = os.path.join(saveDir, "CNN_denoising_tests.zip")

# Parameters
kernel_s = 5
# poolSize = 2
# numKernels = int(sys.argv[4])
batch_size = 32
epochs = 100
imageSize = 128

In [5]:
def extract_train(inputZip,imageSize):
    with zipfile.ZipFile(inputZip, mode="r") as archive:
        # Separate files into train, validation and test
        train_set, test_set = train_test_split(archive.namelist()[0:100], test_size=0.3, random_state=0)
        val_set, test_set = train_test_split(test_set, test_size=0.5, random_state=0)
        train = np.zeros([len(train_set), imageSize, imageSize, 3])
        val = np.zeros([len(val_set), imageSize, imageSize, 3])

        for idx, filename in enumerate(train_set):
            data = archive.read(filename)
            image = cv2.imdecode(np.frombuffer(data, np.uint8), 1)
            train[idx,:,:,:] = image.astype('float32')/255.0

        for idx, filename in enumerate(val_set):
            data = archive.read(filename)
            image = cv2.imdecode(np.frombuffer(data, np.uint8), 1)
            val[idx,:,:,:] = image.astype('float32')/255.0
            
    return train, test_set, val

def extract_test(inputZip,test_set):
    with zipfile.ZipFile(inputZip, mode="r") as archive:
        
        test = np.zeros([len(test_set), imageSize, imageSize, 3])

        for idx, filename in enumerate(test_set):
            data = archive.read(filename)
            image = cv2.imdecode(np.frombuffer(data, np.uint8), 1)
            test[idx,:,:,:] = image.astype('float32')/255.0

    return test

In [6]:
X_train, X_test_set, X_val = extract_train(inputZip_blur, imageSize)

# y_train, y_test_set, y_val = extract_train(inputZip_origin, imageSize)

In [8]:
X_train.shape

(70, 128, 128, 3)